In [1]:
!git clone 'https://github.com/Shenggan/BCCD_Dataset.git'

import os, sys, random, shutil
import xml.etree.ElementTree as ET
from glob import glob
import pandas as pd
from shutil import copyfile
import pandas as pd
from sklearn import preprocessing, model_selection
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import patches
import numpy as np

annotations = sorted(glob('/content/BCCD_Dataset/BCCD/Annotations/*.xml'))

df = []
cnt = 0
for file in annotations:
  prev_filename = file.split('/')[-1].split('.')[0] + '.jpg'
  filename = str(cnt) + '.jpg'
  row = []
  parsedXML = ET.parse(file)
  for node in parsedXML.getroot().iter('object'):
    blood_cells = node.find('name').text
    xmin = int(node.find('bndbox/xmin').text)
    xmax = int(node.find('bndbox/xmax').text)
    ymin = int(node.find('bndbox/ymin').text)
    ymax = int(node.find('bndbox/ymax').text)

    row = [prev_filename, filename, blood_cells, xmin, xmax, ymin, ymax]
    df.append(row)
  cnt += 1

data = pd.DataFrame(df, columns=['prev_filename', 'filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax'])

data[['prev_filename','filename', 'cell_type', 'xmin', 'xmax', 'ymin', 'ymax']].to_csv('/content/blood_cell_detection.csv', index=False)
print(data.head(10))

img_width = 640
img_height = 480

def width(df):
  return int(df.xmax - df.xmin)
def height(df):
  return int(df.ymax - df.ymin)
def x_center(df):
  return int(df.xmin + (df.width/2))
def y_center(df):
  return int(df.ymin + (df.height/2))
def w_norm(df):
  return df/img_width
def h_norm(df):
  return df/img_height

df = pd.read_csv('/content/blood_cell_detection.csv')

le = preprocessing.LabelEncoder()
le.fit(df['cell_type'])
print(le.classes_)
labels = le.transform(df['cell_type'])
df['labels'] = labels

df['width'] = df.apply(width, axis=1)
df['height'] = df.apply(height, axis=1)

df['x_center'] = df.apply(x_center, axis=1)
df['y_center'] = df.apply(y_center, axis=1)

df['x_center_norm'] = df['x_center'].apply(w_norm)
df['width_norm'] = df['width'].apply(w_norm)

df['y_center_norm'] = df['y_center'].apply(h_norm)
df['height_norm'] = df['height'].apply(h_norm)

print(df.head(30))

df_train, df_valid = model_selection.train_test_split(df, test_size=0.1, random_state=13, shuffle=True)
print(df_train.shape, df_valid.shape)

os.mkdir('/content/bcc/')
os.mkdir('/content/bcc/images/')
os.mkdir('/content/bcc/images/train/')
os.mkdir('/content/bcc/images/valid/')

os.mkdir('/content/bcc/labels/')
os.mkdir('/content/bcc/labels/train/')
os.mkdir('/content/bcc/labels/valid/')

def segregate_data(df, img_path, label_path, train_img_path, train_label_path):
  filenames = []
  for filename in df.filename:
    filenames.append(filename)
  filenames = set(filenames)

  for filename in filenames:
    yolo_list = []

    for _,row in df[df.filename == filename].iterrows():
      yolo_list.append([row.labels, row.x_center_norm, row.y_center_norm, row.width_norm, row.height_norm])

    yolo_list = np.array(yolo_list)
    txt_filename = os.path.join(train_label_path,str(row.prev_filename.split('.')[0])+".txt")
    # Save the .img & .txt files to the corresponding train and validation folders
    np.savetxt(txt_filename, yolo_list, fmt=["%d", "%f", "%f", "%f", "%f"])
    shutil.copyfile(os.path.join(img_path,row.prev_filename), os.path.join(train_img_path,row.prev_filename))

## Apply function ##
src_img_path = "/content/BCCD_Dataset/BCCD/JPEGImages/"
src_label_path = "/content/BCCD_Dataset/BCCD/Annotations/"

train_img_path = "/content/bcc/images/train"
train_label_path = "/content/bcc/labels/train"

valid_img_path = "/content/bcc/images/valid"
valid_label_path = "/content/bcc/labels/valid"

segregate_data(df_train, src_img_path, src_label_path, train_img_path, train_label_path)
segregate_data(df_valid, src_img_path, src_label_path, valid_img_path, valid_label_path)

print("No. of Training images", len(os.listdir('/content/bcc/images/train')))
print("No. of Training labels", len(os.listdir('/content/bcc/labels/train')))

print("No. of valid images", len(os.listdir('/content/bcc/images/valid')))
print("No. of valid labels", len(os.listdir('/content/bcc/labels/valid')))

Cloning into 'BCCD_Dataset'...
remote: Enumerating objects: 800, done.
remote: Total 800 (delta 0), reused 0 (delta 0), pack-reused 800 (from 1)
Receiving objects: 100% (800/800), 7.39 MiB | 22.38 MiB/s, done.
Resolving deltas: 100% (378/378), done.
          prev_filename filename cell_type  xmin  xmax  ymin  ymax
0  BloodImage_00000.jpg    0.jpg       WBC   260   491   177   376
1  BloodImage_00000.jpg    0.jpg       RBC    78   184   336   435
2  BloodImage_00000.jpg    0.jpg       RBC    63   169   237   336
3  BloodImage_00000.jpg    0.jpg       RBC   214   320   362   461
4  BloodImage_00000.jpg    0.jpg       RBC   414   506   352   445
5  BloodImage_00000.jpg    0.jpg       RBC   555   640   356   455
6  BloodImage_00000.jpg    0.jpg       RBC   469   567   412   480
7  BloodImage_00000.jpg    0.jpg       RBC     1    87   333   437
8  BloodImage_00000.jpg    0.jpg       RBC     4    95   406   480
9  BloodImage_00000.jpg    0.jpg       RBC   155   247    74   174
['Platelets' 

In [2]:
!git clone https://github.com/THU-MIG/yolov10.git
!pip install -qr /content/yolov10/requirements.txt

Cloning into 'yolov10'...
remote: Enumerating objects: 20332, done.
remote: Counting objects: 100% (2446/2446), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 20332 (delta 2317), reused 2199 (delta 2198), pack-reused 17886 (from 1)
Receiving objects: 100% (20332/20332), 11.14 MiB | 25.82 MiB/s, done.
Resolving deltas: 100% (14337/14337), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 22.4 MB/s eta 0:0

In [3]:
!mkdir -p /content/weights
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P /content/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!printf "train: /content/bcc/images/train\nval: /content/bcc/images/valid\n\nnc: 3\nnames: ['Platelets', 'RBC', 'WBC']" > bcc.yaml
!pip install -e /content/yolov10/

Obtaining file:///content/yolov10
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ultralytics (pyproject.toml) ... done
  Created wheel for ultralytics: filename=ultralytics-8.1.34-0.editable-py3-none-any.whl size=20548 sha256=19ff1457d3c0a56eed1ae7e9bd1beb7476944188454774077bf84c8f54a05f78
  Stored in directory: /tmp/pip-ephem-wheel-cache-l1akgswz/wheels/51/93/e8/22d2e815ced343915c15d86b2a00d95eb0a997d012527fbea7
Successfully built ultralytics


In [6]:
import os
os.environ['WANDB_MODE'] = 'disabled'

!yolo task=detect mode=train epochs=10 batch=32 plots=True \
model=/content/weights/yolov10n.pt \
data=/content/bcc.yaml name=BCCM save=True augment=False

New https://pypi.org/project/ultralytics/8.3.23 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/weights/yolov10n.pt, data=/content/bcc.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, val_period=1, cache=False, device=None, workers=8, project=None, name=BCCM2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=No

In [ ]:
!yolo task=detect mode=predict save=True \
source=/content/bcc/images/valid \
model=/content/yolov10/runs/detect/BCCM/weights/best.pt \
name=predictmodels

Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.0.1+cu117 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10n summary (fused): 285 layers, 2695586 parameters, 0 gradients, 8.2 GFLOPs

image 1/270 /content/bcc/images/valid/BloodImage_00000.jpg: 480x640 7 RBCs, 1 WBC, 58.2ms
image 2/270 /content/bcc/images/valid/BloodImage_00001.jpg: 480x640 11 RBCs, 1 WBC, 9.0ms
image 3/270 /content/bcc/images/valid/BloodImage_00002.jpg: 480x640 4 RBCs, 8.6ms
image 4/270 /content/bcc/images/valid/BloodImage_00003.jpg: 480x640 1 Platelets, 5 RBCs, 8.4ms
image 5/270 /content/bcc/images/valid/BloodImage_00005.jpg: 480x640 3 Plateletss, 12 RBCs, 8.2ms
image 6/270 /content/bcc/images/valid/BloodImage_00007.jpg: 480x640 1 Platelets, 7 RBCs, 8.6ms
image 7/270 /content/bcc/images/valid/BloodImage_00008.jpg: 480x640 8 RBCs, 8.0ms
image 8/270 /content/bcc/images/valid/BloodImage_00009.jpg: 480x640 1 Platelets, 8 RBCs, 8.8ms
image 9/270 /content/bcc/images/valid/BloodImage_00010.jpg: 480x640 6 RBCs, 1 WBC, 8.1ms
image 10/270 /c

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!zip -r /content/drive/MyDrive/25b.zip /content/yolov10/runs/detect/BCCM2
!zip -r /content/drive/MyDrive/25p.zip /content/yolov10/runs/detect/predictmodels

  adding: content/yolov10/runs/detect/BCCM2/ (stored 0%)
  adding: content/yolov10/runs/detect/BCCM2/args.yaml (deflated 52%)
  adding: content/yolov10/runs/detect/BCCM2/val_batch2_labels.jpg (deflated 12%)
  adding: content/yolov10/runs/detect/BCCM2/confusion_matrix.png (deflated 30%)
  adding: content/yolov10/runs/detect/BCCM2/train_batch230.jpg (deflated 11%)
  adding: content/yolov10/runs/detect/BCCM2/R_curve.png (deflated 12%)
  adding: content/yolov10/runs/detect/BCCM2/F1_curve.png (deflated 15%)
  adding: content/yolov10/runs/detect/BCCM2/train_batch0.jpg (deflated 4%)
  adding: content/yolov10/runs/detect/BCCM2/val_batch2_pred.jpg (deflated 5%)
  adding: content/yolov10/runs/detect/BCCM2/val_batch0_labels.jpg (deflated 12%)
  adding: content/yolov10/runs/detect/BCCM2/val_batch1_labels.jpg (deflated 12%)
  adding: content/yolov10/runs/detect/BCCM2/events.out.tfevents.1726231753.0c89a79b70c6.3115.0 (deflated 92%)
  adding: content/yolov10/runs/detect/BCCM2/weights/ (stored 0%)
  